### AR Modeling

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim

In [5]:
class ARModel(nn.Module):
    def __init__(self, input_size):
        super(ARModel, self).__init__()
        self.linear = nn.Linear(input_size, 1)  # Linear layer: input_size -> 1 (predict the next value)

    def forward(self, x):
        out = self.linear(x)  # x is the past 'p' values
        return out

### Data Gen

In [10]:
import numpy as np

# Generate synthetic data (e.g., a simple sine wave)
time_series = np.sin(np.linspace(0, 100, 1000))

# Prepare input-output pairs for the AR model
def create_dataset(series, p):
    X, y = [], []
    for i in range(len(series) - p):
        X.append(series[i:i + p])
        y.append(series[i + p])
    return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

# Hyperparameters
p = 5  # Number of lagged time steps (AR(p) model)

# Create the dataset
X, y = create_dataset(time_series, p)


In [11]:
X.shape

torch.Size([995, 5])

#### Train

In [12]:
# Define the AR model
input_size = p
model = ARModel(input_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
num_epochs = 200
for epoch in range(num_epochs):
    model.train()
    
    # Zero gradients
    optimizer.zero_grad()
    
    # Forward pass: Get the model's prediction
    output = model(X)
    
    # Compute the loss
    loss = criterion(output.squeeze(), y)
    
    # Backward pass: Compute the gradients
    loss.backward()
    
    # Update the parameters
    optimizer.step()
    
    if (epoch + 1) % 20 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [20/200], Loss: 0.0541
Epoch [40/200], Loss: 0.0524
Epoch [60/200], Loss: 0.0429
Epoch [80/200], Loss: 0.0364
Epoch [100/200], Loss: 0.0306
Epoch [120/200], Loss: 0.0252
Epoch [140/200], Loss: 0.0205
Epoch [160/200], Loss: 0.0163
Epoch [180/200], Loss: 0.0128
Epoch [200/200], Loss: 0.0099


### Eval & Pred

In [13]:
# Evaluation mode
model.eval()

# Make predictions
with torch.no_grad():
    # Test the model on the last 'p' values of the time series
    last_values = torch.tensor(time_series[-p:], dtype=torch.float32).unsqueeze(0)  # Reshape to (1, p)
    predicted_value = model(last_values)
    
    print(f"Predicted next value: {predicted_value.item()}")

Predicted next value: -0.522720992565155
